In [1]:
import os
import sys

In [2]:
#sys.path.insert(0, '/Users/marusy/Programming/another-fem-framework/')
sys.path.insert(0, '/home/lshtanko/Programming/another-fem-framework/')
#sys.path.insert(0, '/Users/leonshting/Programming/Schlumberger/fem-framework/')

In [3]:
import numpy as np
from common import polynom_factory
import h5py

In [4]:
limits  = (1,10)
orders = range(*limits)
distribution = 'globatto'
dim = 2
standard_size = (0,1)

In [5]:
f = h5py.File('../datasources/{}_{}_{}_matrices.h5'.format(*limits, distribution))

In [6]:
order_group = f.create_dataset('order_range', data=np.array(orders, dtype=np.int32))
MM_group = f.create_group('mass_matrices')
GG_group = f.create_group('grad_matrices')
M_maps_group = f.create_group('M_maps')

In [7]:
for order in orders:
    MM_group.create_dataset(name='base_M_order_{}'.format(order),
                            data=polynom_factory.local_funcfunc_matrix(dim=dim,
                                                                       distribution=distribution,
                                                                       order=order,
                                                                       size=standard_size)
                           )
    print('\r{} order done'.format(order), end=' ')

9 order done 

In [8]:
for order in orders:
    data = polynom_factory.local_gradgrad_matrix(dim=2, 
                                                 distribution=distribution,
                                                 order=order,
                                                 size=standard_size
                                                )
    GG_group.create_dataset(name='M_order_{}'.format(order),
                            data=data[0])
    
    Xs = np.zeros((order + 1)**dim)
    Ys = np.zeros((order + 1)**dim)
    indices = np.zeros((order + 1)**dim)
    
    for num, (k,v) in enumerate(data[1].items()):
        Xs[num] = k[0]
        Ys[num] = k[1]
        indices[num] = v
    M_map = M_maps_group.create_group(name='M_map_{}'.format(order))
    M_map.create_dataset('X', data=Xs)
    M_map.create_dataset('Y', data=Ys)
    M_map.create_dataset('indices', data=indices)
    print('\r{} order done'.format(order), end=' ')

9 order done 

In [9]:
f.close()

In [10]:
f = h5py.File('../datasources/{}_{}_{}_matrices.h5'.format(*limits, distribution))

In [17]:
daata = f['mass_matrices/base_M_order_3'][()]

In [20]:
daata.sum(axis=1)

array([ 0.00694444,  0.03472222,  0.03472222,  0.00694444,  0.03472222,
        0.17361113,  0.17361112,  0.03472222,  0.03472222,  0.17361112,
        0.17361111,  0.03472221,  0.00694444,  0.03472222,  0.03472221,
        0.00694444])